<a href="https://colab.research.google.com/github/fabiobasson/my-repo/blob/main/oficios_qualidade.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! rm -rf camelot
! mkdir -p laudo
! rm -rf Laudo*
! cd laudo

from google.colab import files
uploaded = files.upload()

Saving Laudo_2628256_7_BUZ_33_RJS_22111210.pdf to Laudo_2628256_7_BUZ_33_RJS_22111210.pdf
Saving Laudo_2628563_7_RI_26_BA_221110214.pdf to Laudo_2628563_7_RI_26_BA_221110214.pdf


In [26]:
import warnings
warnings.filterwarnings('always')
warnings.filterwarnings('ignore')

#!git clone https://www.github.com/camelot-dev/camelot
#!cd camelot
!pip install "camelot-py[cv]"

!apt-get install ghostscript

!pip install 'PyPDF2<3.0'
!pip install xlwt

import pandas as pd
import io

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Reading package lists... Done
Building dependency tree       
Reading state information... Done
ghostscript is already the newest version (9.50~dfsg-5ubuntu4.7).
0 upgraded, 0 newly installed, 0 to remove and 34 not upgraded.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [125]:
import camelot
tables = camelot.read_pdf('laudo/Laudo_2628256.pdf',header=None,index=False)
tables

tab = tables[0].df[[1,0,2]].reset_index(drop=True)


In [94]:
tab.info();tab

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14 entries, 0 to 13
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   1       14 non-null     object
 1   0       14 non-null     object
 2   2       14 non-null     object
dtypes: object(3)
memory usage: 464.0+ bytes


,1,0,2
0,,3. RELAÇÃO DE DADOS RECEBIDOS,
1,Norma de Referência:,Tipos de Dados:,Situação dos Dados no BDEP*:
2,RANP n° 880/2022,3.1. Arquivos Auxiliares - Esquemas de Ferrame...,NÃO CONFORME
3,RANP n° 880/2022,3.2. Arquivos Auxiliares - Dados Direcionais,APROVADO
4,RANP n° 880/2022,3.3. Arquivos Auxiliares - Relatório de Campo ...,
5,RANP n° 880/2022,3.4. Perfis Durante a Perfuração,NÃO CONFORME
6,RANP n° 880/2022,3.5. Perfis Convencionais,APROVADO
7,RANP n° 880/2022,3.6. Perfis Especiais - Teste de Formação a Cabo,APROVADO
8,RANP n° 880/2022,3.7. Perfis Especiais - Sísmica de Poço,
9,RANP n° 880/2022,3.8. Perfis Especiais - Amostrador Lateral,APROVADO


In [95]:
tab = tab.drop(0, axis='index')

In [96]:
tab = tab.drop(1, axis='index')

In [124]:
tab.rename(columns = {1:' Norma Referencia',0:'Tipo Dados',2:'Situacao BDIEP'}, inplace = True)
tab

,Norma Referencia,tipo dados,situacao
2,RANP n° 880/2022,3.1. Arquivos Auxiliares - Esquemas de Ferrame...,NÃO CONFORME
3,RANP n° 880/2022,3.2. Arquivos Auxiliares - Dados Direcionais,APROVADO
4,RANP n° 880/2022,3.3. Arquivos Auxiliares - Relatório de Campo ...,
5,RANP n° 880/2022,3.4. Perfis Durante a Perfuração,NÃO CONFORME
6,RANP n° 880/2022,3.5. Perfis Convencionais,APROVADO
7,RANP n° 880/2022,3.6. Perfis Especiais - Teste de Formação a Cabo,APROVADO
8,RANP n° 880/2022,3.7. Perfis Especiais - Sísmica de Poço,
9,RANP n° 880/2022,3.8. Perfis Especiais - Amostrador Lateral,APROVADO
10,RANP n° 880/2022,3.9. Perfil Digital Processado,NÃO CONFORME
11,RANP n° 880/2022,3.10. Perfil Composto,


In [35]:
descproblema = ['Valor associado a menmonicos'],['Dados nao remetidos ao BDEP'],['Ausência de informação de perflagem conforme NPR '],['Valor inválido associado a mnemônicos '],['Cabeçalho '],['Valor inválido associado a mnemônicos'],['Ausência de informação de perflagem conforme NPR']
df3 = pd.DataFrame(descproblema,columns=['Descrição do Problema'])

df3 = df3.reset_index(drop=True)
df3.info()


#df3.rename(columns = {'16':'team_name', '17':'points_scored'}, inplace = True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7 entries, 0 to 6
Data columns (total 1 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   Descrição do Problema  7 non-null      object
dtypes: object(1)
memory usage: 184.0+ bytes


In [99]:
!pip install pdfminer.six
from pdfminer.high_level import extract_text
import pandas as pd
import io

text  = extract_text('laudo/Laudo_2628256.pdf', 'rb')

df2 = pd.read_fwf(io.StringIO(text))

df2 = df2[16:22]
df2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


,"Agencia Nacional de Petróleo, Gás Natura e Biocombustíveis"
16,2211.1.0210
17,18/11/2022
18,7-BUZ-33-RJS
19,74316029460
20,13/03/2020
21,Petrobras


In [116]:
df2.rename(columns = {'Agencia Nacional de Petróleo, Gás Natura e Biocombustíveis':' CABEÇALHO'}, inplace = True)
df2

,CABEÇALHO
16,2211.1.0210
17,18/11/2022
18,7-BUZ-33-RJS
19,74316029460
20,13/03/2020
21,Petrobras


In [117]:
concat = pd.concat([df2,tab])
concat


,CABEÇALHO,1,0,2
16,2211.1.0210,NaN,NaN,NaN
17,18/11/2022,NaN,NaN,NaN
18,7-BUZ-33-RJS,NaN,NaN,NaN
19,74316029460,NaN,NaN,NaN
20,13/03/2020,NaN,NaN,NaN
21,Petrobras,NaN,NaN,NaN
2,NaN,RANP n° 880/2022,3.1. Arquivos Auxiliares - Esquemas de Ferrame...,NÃO CONFORME
3,NaN,RANP n° 880/2022,3.2. Arquivos Auxiliares - Dados Direcionais,APROVADO
4,NaN,RANP n° 880/2022,3.3. Arquivos Auxiliares - Relatório de Campo ...,
5,NaN,RANP n° 880/2022,3.4. Perfis Durante a Perfuração,NÃO CONFORME


In [108]:
concat = concat.dropna()
concat

,CABEÇALHO,1,0,2


In [110]:
concat.to_excel('file_name.xls')

In [ ]:
df2


,16,17,18,19,20,21
"Agencia Nacional de Petróleo, Gás Natura e Biocombustíveis",2211.1.0210,18/11/2022,7-BUZ-33-RJS,74316029460,13/03/2020,Petrobras


In [ ]:
print(tables[0].parsing_report)
>>> {'accuracy': 100.0, 'whitespace': 0.0, 'order': 1, 'page': 1}
for i in range(tables.n):
    print(tables[i].parsing_report)
>>> {'accuracy': 100.0, 'whitespace': 0.0, 'order': 1, 'page': 1}
>>> {'accuracy': 100.0, 'whitespace': 0.0, 'order': 2, 'page': 1}

print(tables[0])

for i in range(tables.n):
    print(tables[i])

{'accuracy': 100.0, 'whitespace': 37.5, 'order': 1, 'page': 1}
{'accuracy': 100.0, 'whitespace': 37.5, 'order': 1, 'page': 1}
<Table shape=(14, 4)>
<Table shape=(14, 4)>


In [ ]:
!pip install pdfplumber -q
!pip install os -q

import os, pdfplumber

Arquivos= os.listdir()

pdf = pdfplumber.open('laudo/Laudo_2628256_7_BUZ_33_RJS_22111210.pdf')

pdf.pages

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.1/46.1 kB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 33.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.6/143.6 kB 12.4 MB/s eta 0:00:00
ERROR: Could not find a version that satisfies the requirement os (from versions: none)
ERROR: No matching distribution found for os


[<Page:1>, <Page:2>, <Page:3>, <Page:4>]

In [ ]:
!pip install pdfminer.six
from pdfminer.high_level import extract_text
import pandas as pd
import io

text  = extract_text('laudo/Laudo_2628256_7_BUZ_33_RJS_22111210.pdf', 'rb')
print(text)
df2 = pd.read_fwf(io.StringIO(text))


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Agencia Nacional de Petróleo, Gás Natura e Biocombustíveis
Superintendência de Dados Técnicos
Banco de Dados de Exploração e Produção
Laudo de Avaliação de Dados de Poço

1. CABEÇALHO

1.1. Nº do Laudo 
de Reprovação:

1.2. Data da 
Avaliação:

1.3. Nome do 
Poço:

1.4. Nº Cadastro do 
Poço:

1.5. Conclusão do 
Poço:

1.6. Empresa 
Operadora:

2211.1.0210

18/11/2022

7-BUZ-33-RJS

74316029460

13/03/2020

Petrobras

2. CONTROLE DE REMESSAS (USO INTERNO)
2.1. Número de Protocolo da(s) 
Remessa(s):

2.2. Data de Protocolo da(s) 
Remessa(s):

48610.207606/2020-74
 48610.206434/2020-11

22/05/2020
30/04/2020

2.3. Código de Cadastro da(s) 
Mídia(s) no BDEP:

301_2020
254_2020

3. RELAÇÃO DE DADOS RECEBIDOS

Tipos de Dados:

Norma de Referência:

Situação dos Dados no BDEP*:

3.1. Arquivos Auxiliares - Esquemas de Ferramentas

RANP n° 880/2022

NÃO CONFORME

3.2. Arquivos Auxiliares - Dados D

In [ ]:
df2.loc[107:210]

,"Agencia Nacional de Petróleo, Gás Natura e Biocombustíveis"
107,Itens:
108,5.1. Remessa de Dados
109,5.2. Estrutura de Diretórios
110,5.3. Formato / Nomenclatura
111,Não Conformidades:
...,...
206,"os arquivos deste conjunto de dados ao BDEP, a..."
207,880 de 2022.
208,Os dados de PERFIS DURANTE A PERFURAÇÃO ...
209,BRASIL estão avaliados como NÃO CONFORMES...


In [ ]:
df2.loc[107:210].to_excel("texto1.xls")

In [ ]:
search_for = "NÃO CONFORMIDADES" # What to search in the log lines...
line_num = 0 # Line number
lines_found = 0 # Quantity of lines found...
out_file = "saida.txt"
in_file = "texto.txt"

with open(out_file, 'w') as out_file:
    with open(in_file, "r") as in_file:
        for line in in_file:
            line_num += 1
            if search_for in line:
                lines_found += 1
                print("Found '{}' in line {}...".format(search_for, line_num))
                out_file.write(line)

        print("Found {} lines...".format(lines_found))

Found 'NÃO CONFORMIDADES' in line 107...
Found 'NÃO CONFORMIDADES' in line 114...
Found 'NÃO CONFORMIDADES' in line 122...
Found 'NÃO CONFORMIDADES' in line 149...
Found 'NÃO CONFORMIDADES' in line 186...
Found 5 lines...


In [ ]:
file = open('texto.txt')
content = file.readlines() 
content[105:114]


['104,Page 1 of 4\n',
 '105,"\x0c5. NÃO CONFORMIDADES REFERENTES À REMESSA DE DADOS, À EST"\n',
 '106,NOMENCLATURA DOS ARQUIVOS\n',
 '107,Itens:\n',
 '108,5.1. Remessa de Dados\n',
 '109,5.2. Estrutura de Diretórios\n',
 '110,5.3. Formato / Nomenclatura\n',
 '111,Não Conformidades:\n',
 '112,6. NÃO CONFORMIDADES REFERENTES AOS RELATÓRIOS E ARQUIVOS\n']

In [ ]:
text.split('\n')[92:186]

['3.6. Perfis Especiais - Teste de Formação a Cabo',
 '',
 'RANP n° 880/2022',
 '',
 'APROVADO',
 '',
 '3.7. Perfis Especiais - Sísmica de Poço',
 '',
 'RANP n° 880/2022',
 '',
 '3.8. Perfis Especiais - Amostrador Lateral',
 '',
 'RANP n° 880/2022',
 '',
 'APROVADO',
 '',
 '3.9. Perfil Digital Processado',
 '',
 '3.10. Perfil Composto',
 '',
 'RANP n° 880/2022',
 '',
 'NÃO CONFORME',
 '',
 'RANP n° 880/2022',
 '',
 '3.11. Perfil de Acompanhamento Geológico (PAG) - Anexos I, II e III',
 '',
 'RANP n° 880/2022',
 '',
 '3.12. Teste de Formação',
 '',
 'ANP08',
 '',
 '*Legenda da Situação dos Dados de Poço no BDEP:',
 '',
 'NÃO RECEBIDO = Conjunto de dados adquiridos pela operadora, conforme Notificação de Perfilagem Realizada (NPR), mas não ',
 'remetidos ao BDEP;',
 '',
 'NÃO CONFORME = Conjunto de dados avaliados como não conformes. As não conformidades estão descritas neste laudo;',
 '',
 'APROVADO = Conjunto de dados aprovados e carregados no banco de dados. ',
 '',
 '4. RELAÇÃO DOS A

In [ ]:
!pip install tabula-py
!pip install tabulate

from tabula import read_pdf
from tabulate import tabulate
import pandas as pd
import io

# Read the only the page n°6 of the file
text = read_pdf('laudo/Laudo_2628256_7_BUZ_33_RJS_22111210.pdf',pages = 0, 
                         multiple_tables = True, stream = True, encoding="utf-8")

# Transform the result into a string table format
table = tabulate(text)

# Transform the table into dataframe
df1 = pd.read_fwf(io.StringIO(table))
df1
# Save the final result as excel file
#df1[3:11].to_excel("oficios_qualidade1.xls")

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 66.1 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


,---------------------------------,----------------------------------,-----------------------------------,-------------------------------,-------------------------------.1,-------------------------------.2,----------------------------------------------------------,-------------------------------------------------------,-------------------------------------------------------.1
0,0 1.1. No do Laudo,0 1.2. Data da,0 1.3. Nome do,0 1.4. No Cadastro do,0 1.5. Conclusão do,0 1.6. Empresa,NaN,NaN,NaN
1,1 de Reprovação:,1 Avaliação:,1 Poço:,1 Poço:,1 Poço:,1 Operadora:,NaN,NaN,NaN
2,2 2211.1.0210,2 18/11/2022,2 7-BUZ-33-RJS,2 74316029460,2 13/03/2020,2 Petrobras,NaN,NaN,NaN
3,"Name: 1. CABEÇALHO, dtype: object","Name: Unnamed: 0, dtype: object","Name: Unnamed: 1, dtype: object","Name: Unnamed: 2, dtype: object","Name: Unnamed: 3, dtype: object","Name: Unnamed: 4, dtype: object",NaN,NaN,NaN
4,NaN,0 2.3. Código de Cadastro da(s),NaN,NaN,NaN,NaN,0 2.1. Número de Protocolo da(s) 2.2. Data ...,NaN,NaN
5,NaN,1 Mídia(s) no BDEP:,NaN,NaN,NaN,NaN,1 Remessa(s): Rem...,NaN,NaN
6,NaN,"Name: Unnamed: 0, dtype: object",NaN,NaN,NaN,NaN,"Name: 2. CONTROLE DE REMESSAS (USO INTERNO), d...",NaN,NaN
7,NaN,0 Situação dos Dados no BDEP*:,NaN,NaN,NaN,NaN,NaN,0 Tipos de Dados: Norma de Re...,NaN
8,NaN,1 NÃO CONFORME,NaN,NaN,NaN,NaN,NaN,1 3.1. Arquivos Auxiliares - Esquemas de F...,NaN
9,NaN,2 APROVADO,NaN,NaN,NaN,NaN,NaN,2 3.2. Arquivos Auxiliares - Dados Direcio...,NaN


In [ ]:
import PyPDF2

pdf_file = open('laudo/Laudo_3047648.pdf','rb')

dados_pdf = PyPDF2.PdfFileReader(pdf_file)

pagina1 = dados_pdf.getPage(0)
texto_pagina1 = pagina1.extractText()

df2 = pd.read_fwf(io.StringIO(texto_pagina1))

print(texto_pagina1[180:228])

print(texto_pagina1[229:378])

9 7-MRO-6DB-RJS 74316029648 23/02/2021 Petrobras
.1. Nº do Laudo 
de Reprovação:1.2. Data da 
Avaliação :1.3. Nome do 
Poço:1.4. Nº Cadastro do 
Poço:1.5. Conclusão do 
Poço:1.6. Empresa 
Operadora:


In [ ]:
print(dados_pdf.getPage(0))

{'/Type': '/Page', '/Parent': IndirectObject(2, 0, 139966990663536), '/Resources': IndirectObject(6, 0, 139966990663536), '/Contents': [IndirectObject(5, 0, 139966990663536)], '/MediaBox': [0, 0, 594, 842], '/CropBox': [0, 0, 594, 842]}


In [ ]:
page = pdf.pages[0]
page

<Page:1>

In [ ]:
tokenized_text = list(text)
print(tokenized_text)

['A', 'g', 'e', 'n', 'c', 'i', 'a', ' ', 'N', 'a', 'c', 'i', 'o', 'n', 'a', 'l', ' ', 'd', 'e', ' ', 'P', 'e', 't', 'r', 'ó', 'l', 'e', 'o', ',', ' ', 'G', 'á', 's', ' ', 'N', 'a', 't', 'u', 'r', 'a', ' ', 'e', ' ', 'B', 'i', 'o', 'c', 'o', 'm', 'b', 'u', 's', 't', 'í', 'v', 'e', 'i', 's', '\n', 'S', 'u', 'p', 'e', 'r', 'i', 'n', 't', 'e', 'n', 'd', 'ê', 'n', 'c', 'i', 'a', ' ', 'd', 'e', ' ', 'D', 'a', 'd', 'o', 's', ' ', 'T', 'é', 'c', 'n', 'i', 'c', 'o', 's', '\n', 'B', 'a', 'n', 'c', 'o', ' ', 'd', 'e', ' ', 'D', 'a', 'd', 'o', 's', ' ', 'd', 'e', ' ', 'E', 'x', 'p', 'l', 'o', 'r', 'a', 'ç', 'ã', 'o', ' ', 'e', ' ', 'P', 'r', 'o', 'd', 'u', 'ç', 'ã', 'o', '\n', 'L', 'a', 'u', 'd', 'o', ' ', 'd', 'e', ' ', 'A', 'v', 'a', 'l', 'i', 'a', 'ç', 'ã', 'o', ' ', 'd', 'e', ' ', 'D', 'a', 'd', 'o', 's', ' ', 'd', 'e', ' ', 'P', 'o', 'ç', 'o', '\n', '\n', '1', '.', ' ', 'C', 'A', 'B', 'E', 'Ç', 'A', 'L', 'H', 'O', '\n', '\n', '1', '.', '1', '.', ' ', 'N', 'º', ' ', 'd', 'o', ' ', 'L', 'a', 'u